Canva color pallettes: 
http://policyviz.com/wp-content/uploads/2017/01/ExcelColorPalettes.zip 

https://policyviz.com/2017/01/10/150-color-palettes-for-excel

In [1]:
import xml.etree.ElementTree as ET
import zipfile
import io

import json

In [2]:
def parse_xml_to_stripped_tree(zf, file):
    """
    open the file from the zip object zf and strip the namespace; return root 
    """
    # https://stackoverflow.com/a/33997423
    with zf.open(file) as f:
        s = f.read()

    it = ET.iterparse(io.StringIO(s.decode('utf-8')))
    for _, el in it:
        if '}' in el.tag:
            el.tag = el.tag.split('}', 1)[1]  # strip all namespaces
        for at in list(el.attrib.keys()): # strip namespaces of attributes too
            if '}' in at:
                newat = at.split('}', 1)[1]
                el.attrib[newat] = el.attrib[at]
                del el.attrib[at]
    root = it.root
    return root 

In [3]:
def extract_colors_from_root(root):
    extracted_colors = []
    for elem in root:
        for c in elem:
            if c.tag=="srgbClr":
                col_dict = {}
                col_dict['tag'] = elem.tag
                c_val = c.attrib['val']

                # sanitize the hex color
                if c_val[0]!='#':
                    c_val = '#'+c_val
                c_val = c_val.strip().lower() 

                col_dict['val'] = c_val 
                extracted_colors.append(col_dict)
    return extracted_colors

In [19]:
def xml_to_color_dict(zf, file):
    root = parse_xml_to_stripped_tree(zf, file)

    pal_name = root.attrib['name'].strip()

    extracted_colors = extract_colors_from_root(root)
    hex_colorlist = [c['val'] for c in extracted_colors]
    
    four_selected_tags = ['accent{}'.format(x) for x in [1,2,3,4]]    
    hex_colorlist_four = [c['val'] for c in extracted_colors if c['tag'] in four_selected_tags]    

    pal_dict = {}
    pal_dict['pallette'] = pal_name
    pal_dict['pallette_camel'] = ''.join([x.replace('-','').title() for x in pal_name.split(' ')])
    pal_dict['extracted_colors'] = extracted_colors
    pal_dict['hex_colors'] = hex_colorlist
    pal_dict['hex_colors4'] = hex_colorlist_four

    return pal_dict

In [20]:
zf = zipfile.ZipFile('ExcelColorPalettes.zip', 'r')
xml_files = []
for name in zf.namelist():
    if all([name.startswith('ExcelColorPalettes'), name.endswith('.xml')]): 
        xml_files.append(name)

In [21]:
pallette_json = []
for t in xml_files:
    pd = xml_to_color_dict(zf, t)
    pallette_json.append(pd)

In [22]:
with open('canva_palletes.json', 'w') as f:
    js = json.dumps(pallette_json)
    f.write(js)